Change the Hardware to GPU (Edit>Notebook Setting>GPU)

Downlaod the packages

In [ ]:
!pip install face_recognition

Run the below commands to create two folders


Upload the images in "Known" folder for scanning
Upload in "Unknown" folder for detection

In [ ]:
!mkdir known
!mkdir unknown

IMAGE DETECTION

In [ ]:
import face_recognition
import cv2
import os
from google.colab.patches import cv2_imshow

def read_img(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found or invalid image file: {path}")
    (h, w) = img.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(img, (width, height))

# Variables to hold known encodings and names
known_encodings = []
known_names = []
known_dir = 'known'

# Process known images
for file in os.listdir(known_dir):
    img_path = os.path.join(known_dir, file)
    if not file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Filter only image files
        print(f"Skipping non-image file: {file}")
        continue

    try:
        img = read_img(img_path)
        encodings = face_recognition.face_encodings(img)

        if encodings:  # Ensure at least one face is detected
            known_encodings.append(encodings[0])
            known_names.append(file.split('.')[0])
    except FileNotFoundError as e:
        print(e)

# Process unknown images
unknown_dir = 'unknown'
for file in os.listdir(unknown_dir):
    print("Processing", file)
    img_path = os.path.join(unknown_dir, file)
    if not file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Filter only image files
        print(f"Skipping non-image file: {file}")
        continue

    try:
        img = read_img(img_path)
        encodings = face_recognition.face_encodings(img)

        if encodings:  # Ensure at least one face is detected
            img_enc = encodings[0]
            face_locations = face_recognition.face_locations(img)  # Get face locations
            results = face_recognition.compare_faces(known_encodings, img_enc)

            for i, match in enumerate(results):
                if match:
                    name = known_names[i]
                    (top, right, bottom, left) = face_locations[0]
                    cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
                    cv2_imshow(img)

            print(results)
    except FileNotFoundError as e:
        print(e)


LIVE DETECTION USING OPEN-CV

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
import base64
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
import time
from google.colab.output import eval_js

# Function to resize image
def read_img(frame):
    (h, w) = frame.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(frame, (width, height))

# Variables for known faces
known_encodings = []
known_names = []
known_dir = 'known'  # Directory where known faces are stored

# Process known images (add known faces)
for file in os.listdir(known_dir):
    img_path = os.path.join(known_dir, file)
    if not file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Filter only image files
        print(f"Skipping non-image file: {file}")
        continue

    try:
        img = read_img(cv2.imread(img_path))
        encodings = face_recognition.face_encodings(img)

        if encodings:  # Ensure at least one face is detected
            known_encodings.append(encodings[0])
            known_names.append(file.split('.')[0])
    except FileNotFoundError as e:
        print(e)

# JavaScript to access webcam and capture frames for OpenCV
def start_video_stream():
    js_code = """
    var video = document.createElement('video');
    video.width = 640;
    video.height = 480;
    video.autoplay = true;
    video.style.display = 'inline-block';
    document.body.appendChild(video);

    navigator.mediaDevices.getUserMedia({ video: true }).then(function (stream) {
        video.srcObject = stream;
    }).catch(function (error) {
        console.error("Error accessing webcam:", error);
        alert("Webcam access failed. Please check your permissions.");
    });
    """
    display(Javascript(js_code))

def capture_frame():
    js_code = """
    async function capture_frame(){
        var video = document.querySelector('video');
        var canvas = document.createElement('canvas');
        canvas.width = 640;
        canvas.height = 480;
        var context = canvas.getContext('2d');
        context.drawImage(video, 0, 0, canvas.width, canvas.height);
        return canvas.toDataURL('image/jpeg');
    }
    capture_frame();
    """
    return eval_js(js_code)

# Start video stream (using JavaScript)
start_video_stream()

# Perform live face recognition
try:
    while True:
        # Capture a frame (base64 encoded image)
        frame_data = capture_frame()

        # Decode the image data from base64
        img_data = base64.b64decode(frame_data.split(",")[1])
        np_img = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(np_img, cv2.IMREAD_COLOR)

        # Perform face recognition
        img = read_img(img)

        # Find face locations and encodings
        face_locations = face_recognition.face_locations(img)
        encodings = face_recognition.face_encodings(img, face_locations)

        # Loop over detected faces
        for (top, right, bottom, left), enc in zip(face_locations, encodings):
            matches = face_recognition.compare_faces(known_encodings, enc)

            name = "Unknown"
            if True in matches:
                first_match_index = matches.index(True)
                name = known_names[first_match_index]

            # Draw rectangle around the face and add the name
            cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 200), 2)
            cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0,0,200), 1)

        # Display the image with face detection
        cv2_imshow(img)

        # Add a small delay to avoid too fast processing
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Face recognition stopped.")
